In [1]:
import tensorflow as tf### models
import numpy as np### math computations
import seaborn as sns### visualizations
import matplotlib.pyplot as plt### plotting bar chart
import datetime
import pathlib
import io
from datetime import datetime
import json
import xml.etree.ElementTree as ET
import os
import shutil
import cv2
import time
import random
from PIL import Image
import albumentations as A
import tensorflow_datasets as tfds
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import (GlobalAveragePooling2D, Activation, MaxPooling2D, Add, Conv2D, MaxPool2D, Dense,
                                     Flatten, InputLayer, BatchNormalization, Input, Embedding, Permute,
                                     Dropout, RandomFlip, RandomRotation, LayerNormalization, MultiHeadAttention,
                                     RandomContrast, Rescaling, Resizing, Reshape, LeakyReLU)
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (Callback, CSVLogger, EarlyStopping, LearningRateScheduler,
                                        ModelCheckpoint, ReduceLROnPlateau)
from tensorflow.keras.regularizers import L2, L1
from tensorflow.keras.initializers import RandomNormal

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.20 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


#Dataset Downlaod

In [2]:
! pip install -q kaggle
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle
! chmod 600 /root/.kaggle/kaggle.json
! kaggle datasets download -d huanghanchina/pascal-voc-2012

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/huanghanchina/pascal-voc-2012
License(s): DbCL-1.0
100% 3.63G/3.63G [01:53<00:00, 43.3MB/s]
100% 3.63G/3.63G [01:53<00:00, 34.5MB/s]


In [3]:
! unzip "/content/pascal-voc-2012.zip" -d "/content/dataset"

Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001829.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001874.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001876.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001882.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001885.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001895.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001896.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001926.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001966.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001971.png  
  inflating: /content/dataset/voc2012/VOC2012/SegmentationClass/2008_001992.png  
  inflating: /content/dataset/voc2012/VOC2012/S

In [4]:
val_list=['2007_000027.jpg','2007_000032.jpg','2007_000033.jpg','2007_000039.jpg','2007_000042.jpg','2007_000061.jpg',
          '2007_000063.jpg','2007_000068.jpg','2007_000121.jpg','2007_000123.jpg','2007_000129.jpg','2007_000170.jpg',
          '2007_000175.jpg','2007_000187.jpg','2007_000241.jpg','2007_000243.jpg','2007_000250.jpg','2007_000256.jpg',
          '2007_000272.jpg','2007_000323.jpg','2007_000332.jpg','2007_000333.jpg','2007_000346.jpg','2007_000363.jpg',
          '2007_000364.jpg','2007_000392.jpg','2007_000423.jpg','2007_000452.jpg','2007_000464.jpg','2007_000480.jpg',
          '2007_000491.jpg','2007_000504.jpg','2007_000515.jpg','2007_000528.jpg','2007_000529.jpg','2007_000549.jpg',
          '2007_000559.jpg','2007_000572.jpg','2007_000584.jpg','2007_000629.jpg','2007_000636.jpg','2007_000645.jpg',
          '2007_000648.jpg','2007_000661.jpg','2007_000663.jpg','2007_000664.jpg','2007_000676.jpg','2007_000713.jpg',
          '2007_000720.jpg','2007_000727.jpg','2007_000733.jpg','2007_000738.jpg','2007_000762.jpg','2007_000768.jpg',
          '2007_000783.jpg','2007_000793.jpg','2007_000799.jpg','2007_000804.jpg','2007_000807.jpg','2007_000822.jpg',
          '2007_001299.jpg','2007_001311.jpg','2007_001321.jpg','2007_001340.jpg']

In [5]:
!mkdir /content/dataset/VOC2012/ValJPEGImages/
!mkdir /content/dataset/VOC2012/ValAnnotations

In [7]:
for name in val_list:
  shutil.move(train_maps+name[:-3]+"xml", val_maps+name[:-3]+"xml")

In [8]:
for name in val_list:
  shutil.move(train_images+name, val_images+name)

#Dataset Preparation

In [6]:
train_images='/content/dataset/VOC2012/JPEGImages/'
train_maps='/content/dataset/VOC2012/Annotations/'

val_images='/content/dataset/VOC2012/ValJPEGImages/'
val_maps='/content/dataset/VOC2012/ValAnnotations/'

classes=['aeroplane','bicycle','bird','boat','bottle','bus','car','cat','chair','cow','diningtable',
         'dog','horse','motorbike','person','pottedplant','sheep','sofa','train','tvmonitor']

B=2
N_CLASSES=len(classes)
H,W =224,224
SPLIT_SIZE=H//32
print(SPLIT_SIZE)
N_EPOCHS=135
BATCH_SIZE=32


7


In [48]:
def preprocess_xml(filename):
  tree = ET.parse(filename)
  root = tree.getroot()
  size_tree = root.find('size')
  height = float(size_tree.find('height').text)
  width = float(size_tree.find('width').text)
  bounding_boxes=[]
  for object_tree in root.findall('object'):
    for bounding_box in object_tree.iter('bndbox'):
      xmin = (float(bounding_box.find('xmin').text))
      ymin = (float(bounding_box.find('ymin').text))
      xmax = (float(bounding_box.find('xmax').text))
      ymax = (float(bounding_box.find('ymax').text))
      break
    class_name = object_tree.find('name').text
    class_dict={classes[i]:i for i in range(len(classes))}
    bounding_box = [
        (xmin+xmax)/(2*width),(ymin+ymax)/(2*height),(xmax-xmin)/width,
        (ymax-ymin)/height,class_dict[class_name]]
    bounding_boxes.append(bounding_box)
  return tf.convert_to_tensor(bounding_boxes)

In [49]:

preprocess_xml(val_maps+"2007_000032.xml")

#getting co-ordinates of bounding boxes wrt whole image with pascal class number

<tf.Tensor: shape=(4, 5), dtype=float32, numpy=
array([[ 0.479     ,  0.4644128 ,  0.542     ,  0.37366548,  0.        ],
       [ 0.33      ,  0.37544483,  0.128     ,  0.12455516,  0.        ],
       [ 0.408     ,  0.727758  ,  0.036     ,  0.17437722, 14.        ],
       [ 0.07      ,  0.7597865 ,  0.036     ,  0.17437722, 14.        ]],
      dtype=float32)>

In [11]:
'''as per pascal paper we need co-ordinates of centroid of bounding boxes wrt to grid cell
in which center of class lies'''

def generate_output(bounding_boxes):
  output_label = tf.Variable(tf.zeros([SPLIT_SIZE,SPLIT_SIZE,N_CLASSES+5]))

  for b in range(len(bounding_boxes)):
    grid_x = bounding_boxes[b][0] *  SPLIT_SIZE     #0.49
    grid_y = bounding_boxes[b][1] *  SPLIT_SIZE     #5.31
    i = int(grid_x)                                 #rounding up to floor 0
    j = int(grid_y)                                 #rounding up to floor 5
                                                    #(i,j) = (0,5) grid

    if (output_label[i,j,0]==0):
      output_label[i,j,0:5].assign([1.,grid_x%1,grid_y%1,bounding_boxes[b][2],bounding_boxes[b][3]])
      output_label[i,j,5+int(bounding_boxes[b][4])].assign(1.0)

  return output_label

In [50]:
def generate_output1(bounding_boxes):
  output_label=np.zeros((SPLIT_SIZE,SPLIT_SIZE,N_CLASSES+5))
  bounding_boxes = np.array(bounding_boxes)
  for b in range(len(bounding_boxes)):
    grid_x=bounding_boxes[...,b,0]*SPLIT_SIZE
    grid_y=bounding_boxes[...,b,1]*SPLIT_SIZE
    i=int(grid_x)
    j=int(grid_y)

    output_label[i,j,0:5]=[1.,grid_x%1,grid_y%1,bounding_boxes[...,b,2],bounding_boxes[...,b,3]]
    output_label[i,j,5+int(bounding_boxes[...,b,4])]=1.

  return tf.convert_to_tensor(output_label,tf.float32)

In [51]:
generate_output1(preprocess_xml(val_maps+"2007_000032.xml"),)[3][3]

<tf.Tensor: shape=(25,), dtype=float32, numpy=
array([1.        , 0.35300002, 0.25088966, 0.542     , 0.37366548,
       1.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ],
      dtype=float32)>

In [52]:
im_paths=[]
xml_paths=[]

val_im_paths=[]
val_xml_paths=[]


for i in os.listdir(train_maps):

  im_paths.append(train_images+i[:-3]+'jpg')
  xml_paths.append(train_maps+i)

for i in os.listdir(val_maps):

  val_im_paths.append(val_images+i[:-3]+'jpg')
  val_xml_paths.append(val_maps+i)

print(len(im_paths),len(xml_paths))
print(len(val_im_paths),len(val_xml_paths))

17061 17061
64 64


In [53]:
train_dataset=tf.data.Dataset.from_tensor_slices((im_paths,xml_paths))
val_dataset=tf.data.Dataset.from_tensor_slices((val_im_paths,val_xml_paths))

In [54]:
for i in val_dataset.take(1):
  print(i)

(<tf.Tensor: shape=(), dtype=string, numpy=b'/content/dataset/VOC2012/ValJPEGImages/2007_000423.jpg'>, <tf.Tensor: shape=(), dtype=string, numpy=b'/content/dataset/VOC2012/ValAnnotations/2007_000423.xml'>)


In [55]:
def get_imbboxes(im_path,xml_path):
  img=tf.io.decode_jpeg(tf.io.read_file(im_path))
  img=tf.cast(tf.image.resize(img, [H,W]),dtype=tf.float32)

  bboxes=tf.numpy_function(func=preprocess_xml, inp=[xml_path], Tout=tf.float32)
  return img,bboxes

In [56]:
train_dataset=train_dataset.map(get_imbboxes)
val_dataset=val_dataset.map(get_imbboxes)

In [57]:
for i,j in train_dataset.skip(2):
  print(i.shape,j)
  break

(224, 224, 3) tf.Tensor(
[[ 0.356       0.503       0.46666667  0.994      14.        ]
 [ 0.10666667  0.689       0.208       0.198       1.        ]], shape=(2, 5), dtype=float32)


In [ ]:
cv2.imwrite('out_1.jpg',np.array(i))

In [58]:
transforms = A.Compose([
    A.Resize(H,W),
    A.RandomCrop(
         width=np.random.randint(int(0.9*W),W),
         height=np.random.randint(int(0.9*H),H), p=0.5),
    A.RandomScale(scale_limit=0.1, interpolation=cv2.INTER_LANCZOS4,p=0.5),
    A.HorizontalFlip(p=0.5,),
    A.Resize(H,W),

], bbox_params=A.BboxParams(format='yolo', ))

In [59]:
def aug_albument(image,bboxes):
  augmented=transforms(image=image,bboxes=bboxes)
  return [tf.convert_to_tensor(augmented["image"],dtype=tf.float32),
          tf.convert_to_tensor(augmented["bboxes"],dtype=tf.float32)]

In [60]:
def process_data(image,bboxes):
    aug= tf.numpy_function(func=aug_albument, inp=[image,bboxes], Tout=(tf.float32,tf.float32))
    return aug[0],aug[1]

In [61]:
train_dataset=train_dataset.map(process_data)

In [62]:
for i,j in train_dataset.skip(2):
  print(i.shape,j)
  break

(224, 224, 3) tf.Tensor(
[[ 0.62502855  0.5         0.4977778   1.         14.        ]
 [ 0.8900254   0.68789095  0.21994922  0.20159996  1.        ]], shape=(2, 5), dtype=float32)


In [ ]:
cv2.imwrite('out_2.jpg',np.array(i))

In [63]:
def preprocess_augment(img,y):
  img = tf.image.random_brightness(img, max_delta=50.)
  img = tf.image.random_saturation(img, lower=0.5, upper=1.5)
  img = tf.image.random_contrast(img, lower=0.5, upper=1.5)
  #img = tf.image.random_hue(img, max_delta=0.5 )
  img = tf.clip_by_value(img, 0, 255)
  labels=tf.numpy_function(func=generate_output1, inp=[y], Tout=(tf.float32))
  return img,labels

In [64]:
def preprocess(img,y):
  img = tf.cast(tf.image.resize(img, size=[H, W]), dtype=tf.float32)

  labels=tf.numpy_function(func=generate_output1, inp=[y], Tout=(tf.float32))
  return img,labels

In [65]:
train_dataset=train_dataset.map(preprocess_augment)
val_dataset=val_dataset.map(preprocess)

In [66]:
train_dataset = (train_dataset.
                 batch(BATCH_SIZE).
                 prefetch(tf.data.AUTOTUNE))


In [67]:
val_dataset = (val_dataset.
                 batch(BATCH_SIZE).
                 prefetch(tf.data.AUTOTUNE))

In [68]:
for i,j in train_dataset.take(1):
  print(i.shape,j)
  break

(32, 224, 224, 3) tf.Tensor(
[[[[0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   ...
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]

  [[0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   ...
   [1.         0.7430001  0.41167682 ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.         ... 0.         0.
    0.        ]]

  [[0.         0.         0.         ... 0.         0.
    0.        ]
   [0.         0.         0.  

#YOLO MODEL

In [69]:
NUM_FILTERS=512
OUTPUT_DIM=N_CLASSES+5*B

In [80]:
#base_model = tf.keras.applications.resnet50.ResNet50(
base_model=tf.keras.applications.efficientnet.EfficientNetB1(
    weights='imagenet',
    input_shape=(H,W,3),
    include_top=False,
)
base_model.trainable=False

27018416/27018416 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [81]:
input_shape = (None,H, W, 3)  # Replace H, W with your actual image dimensions
output_shape = base_model.compute_output_shape(input_shape)

In [82]:
model=tf.keras.Sequential([
  base_model,
  Conv2D(NUM_FILTERS,(3,3), padding = 'same',kernel_initializer='he_normal',),
  BatchNormalization(),
  LeakyReLU(alpha=0.1),

  Conv2D(NUM_FILTERS,(3,3),padding = 'same',kernel_initializer='he_normal',),
  BatchNormalization(),
  LeakyReLU(alpha=0.1),

  Conv2D(NUM_FILTERS,(3,3),padding = 'same',kernel_initializer='he_normal',),
  BatchNormalization(),
  LeakyReLU(alpha=0.1),

  Conv2D(NUM_FILTERS,(3,3),padding = 'same',kernel_initializer='he_normal',),
  LeakyReLU(alpha=0.1),

  Flatten(),

  Dense(NUM_FILTERS,kernel_initializer='he_normal',),
  BatchNormalization(),
  LeakyReLU(alpha=0.1),

  Dropout(0.5),

  Dense(SPLIT_SIZE*SPLIT_SIZE*OUTPUT_DIM,activation='sigmoid'),

  Reshape((SPLIT_SIZE,SPLIT_SIZE,OUTPUT_DIM)),
])
model.build(input_shape) # Explicitly build the model with the input shape
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb1 (Functional)          │ (None, 7, 7, 1280)          │       6,575,239 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 7, 7, 512)           │       5,898,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_8                │ (None, 7, 7, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_10 (LeakyReLU)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ (None, 7, 7, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_9                │ (None, 7, 7, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_11 (LeakyReLU)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ (None, 7, 7, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_10               │ (None, 7, 7, 512)           │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_12 (LeakyReLU)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ (None, 7, 7, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_13 (LeakyReLU)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │      12,845,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_11               │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_14 (LeakyReLU)           │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1470)                │         754,110 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape_2 (Reshape)                  │ (None, 7, 7, 30)            │               0 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 33,161,285 (126.50 MB)

 Trainable params: 26,581,950 (101.40 MB)

 Non-trainable params: 6,579,335 (25.10 MB)

#YOLO Loss

In [83]:
def compute_iou(boxes1, boxes2):
    boxes1_t = tf.stack([boxes1[..., 0] - boxes1[..., 2] / 2.0,
                         boxes1[..., 1] - boxes1[..., 3] / 2.0,
                         boxes1[..., 0] + boxes1[..., 2] / 2.0,
                         boxes1[..., 1] + boxes1[..., 3] / 2.0],
                        axis=-1)

    boxes2_t = tf.stack([boxes2[..., 0] - boxes2[..., 2] / 2.0,
                         boxes2[..., 1] - boxes2[..., 3] / 2.0,
                         boxes2[..., 0] + boxes2[..., 2] / 2.0,
                         boxes2[..., 1] + boxes2[..., 3] / 2.0],
                        axis=-1)
    print("boxes1_t : \n",boxes1_t)
    print("boxes2_t : \n",boxes2_t,"\n")

    lu = tf.maximum(boxes1_t[..., :2], boxes2_t[..., :2])
    rd = tf.minimum(boxes1_t[..., 2:], boxes2_t[..., 2:])

    print("lu : \n",lu)
    print("rd : \n",rd,"\n")
#to get width and height for intersection_area
    intersection = tf.maximum(0.0, rd - lu)
    area_intersection = intersection[..., 0] * intersection[..., 1]

    print("intersection : ",intersection)
    print("inter_square : ",area_intersection,"\n")

    square1 = boxes1[..., 2] * boxes1[..., 3]
    square2 = boxes2[..., 2] * boxes2[..., 3]

    area_union = tf.maximum(square1 + square2 - area_intersection, 1e-10)
    print("area_union" ,area_union )

    return tf.clip_by_value(area_intersection / area_union, 0.0, 1.0)

In [84]:
def difference(x, y):
  # Cast both tensors to the same data type before subtraction
  x = tf.cast(x, dtype=tf.float32)  # or tf.float64, to match y's type
  y = tf.cast(y, dtype=tf.float32)  # or tf.float32, to match x's type
  return tf.reduce_sum(tf.square(y - x))

In [85]:
def yolo_loss(y_true, y_pred):
    target = y_true[..., 0]

    # Rank and Shape Checks
    print("y_true shape:", y_true.shape)
    print("y_pred shape:", y_pred.shape)

    y_true = tf.ensure_shape(y_true, [1, 7, 7, 13]) # Replace with your expected shape
    y_pred = tf.ensure_shape(y_pred, [1, 7, 7, 18])

    ###################### Object Loss
    y_pred_extract = tf.gather_nd(y_pred, tf.where(target[:] == 1))
    y_target_extract = tf.gather_nd(y_true, tf.where(target[:] == 1))

    print("Grid Position:", tf.where(target[:] == 1))
    print("Predicted:", y_pred_extract.shape)
    print("Actual:", y_target_extract.shape)

    rescaler = tf.where(target[:] == 1) * 32
    upscaler_1 = tf.concat([rescaler[:, 1:], tf.zeros([len(rescaler), 2], dtype=tf.int64)], axis=-1)

    # Bounding box scaling
    actual_upscaler = tf.repeat([[32., 32., 224., 224.]], repeats=len(rescaler), axis=0) * tf.cast(y_target_extract[..., 1:5], dtype=tf.float32)
    prediction_upscaler1 = tf.repeat([[32., 32., 224., 224.]], repeats=len(rescaler), axis=0) * tf.cast(y_pred_extract[..., 1:5], dtype=tf.float32)
    prediction_upscaler2 = tf.repeat([[32., 32., 224., 224.]], repeats=len(rescaler), axis=0) * tf.cast(y_pred_extract[..., 6:10], dtype=tf.float32)

    # Compute actual and predicted bounding box positions
    actual_original = tf.cast(upscaler_1, dtype=tf.float32) + actual_upscaler
    prediction_original1 = tf.cast(upscaler_1, dtype=tf.float32) + prediction_upscaler1
    prediction_original2 = tf.cast(upscaler_1, dtype=tf.float32) + prediction_upscaler2

    # Calculate IOU and mask for best-matching prediction
    mask = tf.cast(tf.math.greater(compute_iou(actual_original, prediction_original2),
                                   compute_iou(actual_original, prediction_original1)), dtype=tf.int32)

    ############################# Object and NoObject Loss
    obj_pred = tf.gather_nd(tf.stack([y_pred_extract[..., 0], y_pred_extract[..., 5]], axis=1), tf.stack([tf.range(len(rescaler)), mask], axis=-1))
    object_loss = difference(obj_pred, tf.ones([len(rescaler)], dtype=tf.float32))

    # NoObject loss
    no_object_mask = tf.where(target[:] == 0)
    no_object_loss_1 = difference(y_pred[..., 0][no_object_mask], tf.zeros([len(no_object_mask[0])]))
    no_object_loss_2 = difference(y_pred[..., 5][no_object_mask], tf.zeros([len(no_object_mask[0])]))
    no_object_loss = no_object_loss_1 + no_object_loss_2

    ############################# Class Loss
    y_pred_classes = tf.gather_nd(y_pred[..., 10:], tf.where(target[:] == 1))
    y_true_classes = tf.gather_nd(y_true[..., 5:], tf.where(target[:] == 1))
    class_loss = difference(y_pred_classes, y_true_classes)

    ############################# Bounding Box Loss
    centre_pred = tf.gather_nd(tf.stack([y_pred_extract[..., 1:3], y_pred_extract[..., 6:8]], axis=1), tf.stack([tf.range(len(rescaler)), mask], axis=-1))
    centre_target = tf.gather_nd(y_true[..., 1:3], tf.where(target[:] == 1))
    centre_loss = difference(centre_pred, centre_target)

    size_pred = tf.gather_nd(tf.stack([y_pred_extract[..., 3:5], y_pred_extract[..., 8:10]], axis=1), tf.stack([tf.range(len(rescaler)), mask], axis=-1))
    size_target = tf.gather_nd(y_true[..., 3:5], tf.where(target[:] == 1))
    size_loss = difference(tf.sqrt(tf.abs(size_pred)), tf.sqrt(tf.abs(size_target)))
    box_loss = centre_loss + size_loss

    # Overall loss
    lambda_coord = 5.0
    lambda_no_obj = 0.5
    loss = object_loss + (lambda_no_obj * no_object_loss) + lambda_coord * box_loss + class_loss

    print("Loss:", loss)
    return loss

In [39]:
'''def generate_output2(bounding_boxes):
  output_label=np.zeros((7,7,13))
  bounding_boxes = np.array(bounding_boxes)
  for b in range(len(bounding_boxes)):
    grid_x=bounding_boxes[...,b,0]*SPLIT_SIZE
    grid_y=bounding_boxes[...,b,1]*SPLIT_SIZE
    i=int(grid_x)
    j=int(grid_y)

    output_label[i,j,0:5]=[1.,grid_x%1,grid_y%1,bounding_boxes[...,b,2],bounding_boxes[...,b,3]]
    output_label[i,j,5+int(bounding_boxes[...,b,4])]=1.

  return tf.convert_to_tensor(output_label,tf.float32)

In [35]:
'''y_true=generate_output2([[0.210784,0.616422,0.127451,0.232843,2]])
#y_true=generate_output2([[0.509804,0.411765,0.107843,0.245098,3],
 #                       [0.210784,0.616422,0.127451,0.232843,2]])
y_true=np.expand_dims(y_true,axis=0)
y_pred=np.random.normal(size = (1,7,7,18))

y_pred[0][1][4] = [0.9,0.47,0.31,0.12,0.23,   1.0,0.2,0.6,0.1,0.95,    0.9,0.8,0.2,0.6,0.1,0.5,0.9,0.35]
#y_pred[0][3][2] = [0.3,0.01,0.08,0.11,0.54,   0.98,0.56,0.88,0.1,0.24, 0.09,0.018,0.22,0.16,0.01,0.05,0.99,0.3]

yolo_loss(y_true,y_pred)

y_true shape: (1, 7, 7, 13)
y_pred shape: (1, 7, 7, 18)
Grid Position: tf.Tensor([[0 1 4]], shape=(1, 3), dtype=int64)
Predicted: (1, 18)
Actual: (1, 13)
boxes1_t : 
 tf.Tensor([[ 32.9411   112.00011   61.490128 164.15694 ]], shape=(1, 4), dtype=float32)
boxes2_t : 
 tf.Tensor([[ 27.2       40.799995  49.600002 253.6     ]], shape=(1, 4), dtype=float32) 

lu : 
 tf.Tensor([[ 32.9411  112.00011]], shape=(1, 2), dtype=float32)
rd : 
 tf.Tensor([[ 49.600002 164.15694 ]], shape=(1, 2), dtype=float32) 

intersection :  tf.Tensor([[16.658901 52.15683 ]], shape=(1, 2), dtype=float32)
inter_square :  tf.Tensor([868.8755], shape=(1,), dtype=float32) 

area_union tf.Tensor([5386.8716], shape=(1,), dtype=float32)
boxes1_t : 
 tf.Tensor([[ 32.9411   112.00011   61.490128 164.15694 ]], shape=(1, 4), dtype=float32)
boxes2_t : 
 tf.Tensor([[ 33.600002 112.159996  60.48     163.68    ]], shape=(1, 4), dtype=float32) 

lu : 
 tf.Tensor([[ 33.600002 112.159996]], shape=(1, 2), dtype=float32)
rd : 
 tf.T

InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:GPU:0}} Expected begin, end, and strides to be 1D equal size tensors, but got shapes [1,48,3], [1,48,3], and [1] instead.
	 [[{{node StridedSlice}}]] [Op:StridedSlice] name: strided_slice/

In [95]:
checkpoint_filepath='/content/YOLO/yolo_efficientnet_b1_new.weights.h5'
callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True
)

In [96]:
def scheduler(epoch, lr):
  if epoch < 40:
    return 1e-3
  elif epoch>=40 and epoch<80:
    return 5e-4
  else:
    return 1e-4

In [97]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [98]:
model.compile(
  loss=yolo_loss,
  optimizer=Adam(1e-3),
)

In [99]:
history = final_model.fit(
  train_dataset,
  validation_data=val_dataset,
  verbose=1,
  epochs=135,
  callbacks = [lr_callback,callback]
)

Epoch 1/135


ValueError: as_list() is not defined on an unknown TensorShape.

In [ ]:
model.load_weights(checkpoint_filepath)

In [ ]:
!mkdir outputs/

In [ ]:
COCO_PATH='/content/drive/MyDrive/Bang/coco_images/'

In [ ]:
def model_test(filename):
  try:
    test_path=COCO_PATH+filename

    print(test_path)

    img=cv2.resize(cv2.imread(test_path),(H,W))

    image=tf.io.decode_jpeg(tf.io.read_file(test_path))
    image=tf.image.resize(image, [H,W])

    output=model.predict(np.expand_dims(image, axis = 0))

    THRESH=.25

    object_positions=tf.concat(
        [tf.where(output[...,0]>=THRESH),tf.where(output[...,5]>=THRESH)],axis=0)
    print(object_positions)
    selected_output=tf.gather_nd(output,object_positions)
    print(selected_output)
    final_boxes=[]
    final_scores=[]

    for i,pos in enumerate(object_positions):
      for j in range(2):
        if selected_output[i][j*5]>THRESH:
          output_box=tf.cast(output[pos[0]][pos[1]][pos[2]][(j*5)+1:(j*5)+5],dtype=tf.float32)

          x_centre=(tf.cast(pos[1],dtype=tf.float32)+output_box[0])*32
          y_centre=(tf.cast(pos[2],dtype=tf.float32)+output_box[1])*32

          x_width,y_height=tf.math.abs(H*output_box[2]),tf.math.abs(W*output_box[3])

          x_min,y_min=int(x_centre-(x_width/2)),int(y_centre-(y_height/2))
          x_max,y_max=int(x_centre+(x_width/2)),int(y_centre+(y_height/2))

          if(x_min<=0):x_min=0
          if(y_min<=0):y_min=0
          if(x_max>=W):x_max=W
          if(y_max>=H):y_max=H
          final_boxes.append(
              [x_min,y_min,x_max,y_max,
              str(classes[tf.argmax(selected_output[...,10:],axis=-1)[i]])])
          final_scores.append(selected_output[i][j*5])
    print(final_scores)
    print('finalboxes',final_boxes)
    final_boxes=np.array(final_boxes)

    object_classes=final_boxes[...,4]
    nms_boxes=final_boxes[...,0:4]

    nms_output=tf.image.non_max_suppression(
        nms_boxes,final_scores,max_output_size=100,iou_threshold=0.2,
        score_threshold=float('-inf')
    )
    print(nms_output)

    for i in nms_output:
      cv2.rectangle(
          img,
          (int(final_boxes[i][0]),int(final_boxes[i][1])),
          (int(final_boxes[i][2]),int(final_boxes[i][3])),(0,0,255),1)
      cv2.putText(
          img,
          final_boxes[i][-1],
          (int(final_boxes[i][0]),int(final_boxes[i][1])+15),
          cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(2,225,155),1
          )

    cv2.imwrite('/content/outputs/'+filename[:-4]+'_det'+'.jpg',cv2.resize(img,(384,384)))
  except:
    print("NO object found !!!")

In [ ]:
for filename in os.listdir(COCO_PATH):
  model_test(filename)